In [7]:
influencers_list1=["toumi_lina","raya_bouallegue","marwa_ben_sghaier",
                  "belleetgourmande","douhalaribii","sabrinebelaiech",
                  "zahragrissa","sabrine.sbei","molkamissaoui","bochrachedli",
                  "fatma.bououn","troudibilel","ahmed.ben.hnia","yasmine.khalifa.mansour",
                  "asma_khalil_ghawar","tripstyleblog","onshm","meryem_hmila_life",
                  "sabrinaasaidani","seima_girl","baraa_bouzgarrou","cooking_doctor",
                  "doctor.karim.mechri","nesrine_hitana","saraabdellaoui","budgeteuse.tn",
                  "lady.samara","maryem_ilmabrouk","fatma.elmahjoub",
                  "lynda.hjaiej","maissa.tekaya","mohamed_bouglia",
                  "olfakaabi","mohamedtaherdridi","ahmed.riabi","thedreamerwildandfree",
                  "lynda_toumy","oubay_naffeti","omny_wish","dhekrachebbi",
                  "el9anounji.ftw","chaima_mansourofficial","dr.amine_said",
                  "rayen_bouajaja","ahmed_mak9","nourchenne_cheguenni","oussama.jellali",
                  "coach_oussama_trabelsi","amal.books","wajd_chelly","ouay.cherni",
                  "khoubaib_jebabli","rayhanettee","nawel_bizid","oumaima_trad","abir_abdelmalek_",
                  "beki_ksri","khormologia","areej_ay","sirinabenayed","k__obs","maryemayadi.tn",
                  "tayebsenglish","themyra1","nadabenjeddou","dridighofrane","ghada.bouajaja","maram_jungle","emnatatou.bg",
                  "mak.and.cha","samy.chaffai","geekeuzz","mr_le_psy","oussema_triter","sonia_younsi","hanene_eleuch"
                  ,"malek_ouni1","khawlachedly","mys_journey","yaluna_magazine","el_7arbousha","afefgharbi",
                  "lydiaasli","loujey","marwenebouzayene","salemmonsieur","khaoulaslimaniofficial",
                  "enjymidooz","oumaymabenhafsia4","hamma_stories","aminastaofficial","myriemboukadida",
                  "najlabenabdallah.official","rimelbenna","dorra_jarraya_official","mayssa.ferchichi",
                  "saoussen_masmoudi","bilalmoufokofficiel","ghadamaatouk","sahri.bahri",
                  "nourhen_hourcheni_official","malekoueslati1","rihem_ben_alaya","mamiaziz","ranyatoumi_officia",
                  "hajer_bayou","yasmeenayari_","mouraderouge","souhaiel.ghidaoui","diotisdiaries","sahar_wali_mallouli",
                  "zaza_show_","eyaofficial__","donia_fekii","gafsaouiamine_","amine.khemakhem","eyaa_chelly_","farahbouglia",
                  "marwa.alayet","afefgharbi","kenzatestouri","azzaslimene","azizos_chafchaa__","emnasadfii","sadfishan",
                  "ayoubsadfi"]

influencers_list2=['Dorra zarrouk', 'Manelamara', 'Ghali', 'Beki ksri', 'Hendsabri', 'Dhaferlabidine', 'Aichabahmed', 'Marwa agrebi', 'Latifaofficial', 'Atiaaichaofficial', 'Dareen haddad', 'Sanfara7050', 'Zaza show', 'Samiatrabelsiii', 'Yousr.fg', 'Marambenazizaofficial', 'Aymenabdennour', 'Lobnasediri', 'Maaloul ali', 'Gharbiikarim', 'Samara riahi', 'Zaiemhedi', 'Khaoulaslimaniofficial', 'Samy.chaffai', 'Theswaggmantv', 'Rymooshka', 'Lotfiabdelli', 'Rachabenmaouia', 'Samiramagroun', 'Amira al jaziri', 'Sami.fehri1', 'Bassoum hamraoui', 'Rymsaidi', 'Myriemboukadida', 'Saberrebai', 'Officialnajlaettounsia', 'Nordo officiel', 'Ramla dhouibi', 'A.l.a official', 'Cheb bachir1', 'Oumaymabenhafsia4', 'Bayazardi.officiel', 'Fatma.bououn', 'Raniatoumi official', 'Lety  lo', 'Maroua issa', 'Mastersina official', 'Lady.samara', 'Najlabenabdallah.official', 'Ghada.chaka', 'Lynda toumy', 'Tarek  baalouch', 'Hanen chograni', 'Emnafakher', 'Yassinebengamra', 'Onsjabeur', 'Chirinelajmi', 'Faycellahdiri', 'Mustafa houas', 'Amina.sta1', 'Imen.cherif.officiel', 'Akrammag', 'Midouch talbi', 'Youssef msekni', 'Yosramahnouch', 'Omar lartiste maquilleur', 'Ahmed landolsi', 'Mayssa.ferchichi', 'Asmaothmaniofficial', 'Dali benjemaa', 'Faf.elkadhi', 'Samir elwafi', 'Rapklay bbj', 'Fatmamahjoubofficial', 'Sabrine taleb', 'Salemmonsieur', 'Rimelbenna', 'Mortadha.ftiti', 'Alikaroui', 'Leilabenkhalifaofficial', 'Fatmanasserofficial', 'Nasseem ziadia', 'Jihenemiled', 'Moalla khouloud', 'Najla violette', 'Ferjanisassioff', 'Maryem  sakka', 'Asmafehri officiel', 'Raya bouallegue', 'Sonia younsi', 'Khalil', 'Ghadamaatouk', 'Saraguendogan', 'Si nacer', 'Hanene eleuch', 'Rym ben messaoud', 'Azzaslimene', 'Tanwicha', 'Meriem ben moulehem  malika', 'Juliachaouachi', 'Meriembenhussein', 'Feriel youssef', 'Peekette', 'Oumaima trad', 'Ahmed kafon.officiel', 'Souhir ben amara', 'Wassim.migalo.official', 'Onshm', 'Sahri.bahri', 'Medislambouglia', 'Salma mahjoubi officiel', 'Mohamed amine gara1', 'Rihem ben alaya', 'Hamza belloumi', 'Sarra  cherif', 'Naoufelouertani', 'Ali zitouni', 'Maissa.tekaya', 'Seima girl', 'Eya.naccache', 'Amine.limemmm', 'Kenzatestouri', 'Aminaminouch', 'Yosramassoudiofficial', 'Raniagabsi', 'Rim bl', 'Benalouaneyohan', 'Ammarjemal5', 'Lina khouili', 'Lacrazyrevolution', 'Ghalia.salhi', 'Kaouther.bel.haj', 'Lydiaasli', 'Mariembchaabane', 'Thisizbalti', 'Liliabenaziza', 'Oussema.soulii', 'Ahmed.riabi', 'Habibaghribi', 'Rymabenaissa', 'Mayaofficial maya', 'Toumi lina', 'Belleetgourmande', 'Thevmaker', 'Nour chiba officiel', 'Raoudhaamri nutrition', 'Kamiliayat', 'Malek.ben.saad', 'Artmastaofficiel', 'Chebbialaaofficiel', 'Thew lifestyle', 'Mariem.nourddine', 'Rabaa essefi officiel', 'Lynda.hjaiej', 'Nahdi.dali.officiel', 'Nourasamet', 'Mouraderouge', 'Jebaliaziz', 'Noorelhoodanaoui', 'Raed.fadaa.officiel', 'Yasmeenayari', 'Fatma.zahi', 'Niels aouay', 'Sandra chihaoui', 'Ibaa hamli', 'Nassimraissiofficial', 'Rahma mdallaa', 'Marwen.ratel', 'Marwen.ratel', 'Ilef.b', 'Sanabencheikhlarbi', 'Asmamek', 'Nabil.ben.mesmia', 'Bassem.srarfi officiel19', 'Sandy.aly', 'Med ali tounsi', 'Ichraqqamar', 'Horcujet', 'Hachemgaddouch', 'Soniadahmani', 'Steevy aymen', 'Joelamrikani', 'Troudibilel', 'Bilel bejii1', 'Sayayassine', 'Benromdhaneolfa', 'Ghaylen chaaleli', 'Inasaly', 'Diotisdiaries', 'Mariemsabbagh', 'Coucoulesgourmands', 'Nadabenjeddou', 'Tunisian.kitchen', 'Cherihane healthy lifestyle', 'Djtora official', 'Safwenboufarwa', 'Bilelmissaoui', 'Rimbougamra', 'Afefjnifen', 'Tastemysweetlife', 'Hanayaich', 'Itsfarahyasmine', 'Nejibbelkadhi', 'Sofien.bengamra', 'Inesosmani', 'Achref ouerghemmi photographe', 'Kenzaachiha', 'Jannah cuisine', 'Asma ben.othmane', 'K  obs', 'Nada mezni hafaiedh', 'Asma.delice momix', 'Kalthoumhandous', 'Sarra ben hammouda', 'Manel  hamrouni', 'Raouf maher', 'Eyadaghnoujofficial', 'Sami.benabdallah', 'Aziz  bey', 'Amal.amamou', 'Sonia hermassi', 'Azizabouraoui', 'Dracowar official', 'Instaieb', 'Azerzaouali', 'Khawla in the kitchen', 'Khormologia', 'Khawlaboukhris', 'Rochdi belgasmi', 'Radhijaidi', 'Majdouline beauty', 'Abir azl', 'Nessrineesalah', 'Nouhabenhammouda', 'Ymeriah', 'Amal hachana', 'Bassem sellami', 'Ayoub dhouib', 'Rayenyoussefofc', 'Emna marouani', 'Cherifouali', 'Rymhammamy', 'Jaaferguesmi1', 'Marwa.benali.712', 'Inessbousselmi', 'Ksourimaya', 'Iheb msekni 10', 'Bahakefi', 'Cuisine  imen', 'Najla chtourou', 'Hamdi hadda', 'Halima.cocinera', 'Areej ay', 'Nadhir mindfreak', 'Elgeneralofficial', 'Imeneznati', 'Assewrbenmohamed', 'Ihebmbarki24', 'Molka g', 'Hendjebali', 'Oussama  haddadi', 'Jamilachihi', 'Imen mehrzi', 'Slimhal', '50mejri', 'Rami jeridi officiel', 'Macneer', 'Kenzafourati', 'Hela melki official', 'Samy.dridi.officiel', 'Lina ben ali officiel', 'Nour guiga', 'Khemissi imen', 'Rahma benhassen1', 'Raouajemli', 'Amine.chermiti', 'Sabrinetrifi', 'Cherifa naffeti', 'Tunisianofficiel', 'Haithemjouini officiel', 'Diwasweetness', 'Moeztoumiofficial', 'Salma.btk', 'Thequeen nutella', 'Nour gn', 'Lamine belkhodja', 'Yasminenajar', 'Tamaracuisine', 'Sabrine choura', 'Saifkhaoui20', 'Hassendosss', 'Aymenlessigue', 'Yassine.gaidi', 'Wajdi.lakhal', 'Rymmkhouja', 'Sonytta', 'Nejibbelhassen', 'Saadiranya', 'Salma jardak', 'Feriel.maraaoui', 'Salimaammar', 'Chaymadimassi', 'Sandra.aouadi', 'Mehdikallelcouture', 'Bayrem kilani', 'Mohamedfrigui', 'Issam.tn', 'Chekra rammeh', 'Jnaynaa', 'Amamoumolka', 'Wael ben abdallah', 'Mariam al ferjani', 'Fakhet amina', 'Auplaisirdesgourmands', 'Intibelhassen', 'Jessnarjess', 'Issou5', 'Narjessrkaiek', 'Vitalunaspirit', 'Feryel selmi', 'Chouchoutas', 'Rabebhosny', 'Nedra lamloum', 'Discoverwithbina', 'Soumow', 'Lotfiboucedra', 'Wajdicherif', 'Tresors de mimi', 'Drfaresbelhassen', 'Tarabine', 'Marwan.ali.officiel', 'Alamarzougui7', 'Neilouz', 'Spartarami', 'Olfa hammouda', 'Imenbourguiba', 'Mayatouati', 'Jazirimalek', 'Chaima mahmoud', 'Ankh nh', 'Ali macheni', 'Drfediamtaallah', 'Myriam gmati', 'Norchene1', 'Bassem chatteli', 'Rayen  kochbati', 'Mohamed jebali officiel', 'Mohamedkhammessi', 'Fahmi riahi 09', 'Sbyalma', 'Ahlem bsl', 'Journaliste gourmande', 'Dalinsomnia', 'Dorsaf al shaal', 'Amal mamastyle', 'Kenzacaidessebsi', 'Amin hajri', 'Ines boubakri', 'Rines tattouage', 'Mhirsiiiiii', 'Salim chebil', 'Atef dkhili', 'Dhaferyoussef', 'Meskini.amine.28', 'Sheimaoueslati', 'Itebakaichi', 'Ghanem zrelli official', 'Hiba saidane', 'Sayeh mariem', 'Ammouna.369', 'Houda hadhoudda', 'Tunisianyogini', 'Yosra.benaissa', 'Djdanjertn', 'Iskandar maalej', 'Joannabensouissi', 'Anissa daoud', 'Hanen.fehem', 'Naestro officiel', 'Streetfood tunisia', 'Ayarikhaled13', 'Aliastasia', 'Hitunis', 'Sadok yeddes', 'Tuniquotes', 'Idryss.kharroubi', 'Hibatelmoudi', 'Ghadaabderrazek', 'Shootme.tn', 'Litayem zied', 'Arijzaafouri official', 'Amin ben amar', 'Aminabouchiba', 'Aymen mahmoud', 'Enes.amara', 'Hajrimyriam', 'Med amine dj ghost', 'Noureddine gamdou', 'Iamdonymk', 'Hanibahba', 'Naderguirat', 'Fatmabouchiba', 'Realraniatoumi', 'Houcinerabii', 'Massy kacha', 'ZAP', 'Rim mathlouthi369', 'Noor.blue', 'Isl eem007', 'La savoureusee', 'Meriembejaoui340', 'Bejiiheb', 'Sassi events', 'Souad bouabana', 'Farouk ben mustapha', 'Med.chou', 'Jyhen hs', 'Dr.habboub', 'Olfette diaries', 'Miniar.rkhami', 'Cuisineolfa', 'Cuisine a deux', 'Afefgharbi', 'Fakherbeautycenter', 'Sawsen gmati', 'Asmabaklouti', 'Sabrine khalifamansour', 'Jane.torjeman', 'Awesmessaoudi', 'Wajihajendoubi', 'Manelabdelkoui', 'Saydouda', 'Mathlouthi hamza', 'Maryem.hz', 'Hela.soyeh', 'Teyssir ksouri', 'Montassartalbi', 'Asmabenahmedofficiel', 'Hamdi van buuren', 'Sisterss diary', 'Jihed cherni', 'Eyasghir', 'Saadofficielbguir', 'Amanielswissi', 'Abirbennani', 'Yasbouabid', 'Sarah hannachi', 'Ben youssef officiel', 'Hamdiharbaoui9', 'Olfakaabi', 'Amira derouiche', 'Samarmatoussi', 'Aminebensalah', 'Mayssakasbi', 'Wahbi khazri 8', 'Badri anice officiel', 'Ineshicheri', 'Sanayoussefofficial', 'Yassine khenissi', 'Malouchelamis', 'Mehdi r2m officiel', 'Maaloul ali']
for inf in influencers_list1:
    if  not (inf in influencers_list2):
        influencers_list2.append(inf)
print(influencers_list2)
print(len(influencers_list2))

['Dorra zarrouk', 'Manelamara', 'Ghali', 'Beki ksri', 'Hendsabri', 'Dhaferlabidine', 'Aichabahmed', 'Marwa agrebi', 'Latifaofficial', 'Atiaaichaofficial', 'Dareen haddad', 'Sanfara7050', 'Zaza show', 'Samiatrabelsiii', 'Yousr.fg', 'Marambenazizaofficial', 'Aymenabdennour', 'Lobnasediri', 'Maaloul ali', 'Gharbiikarim', 'Samara riahi', 'Zaiemhedi', 'Khaoulaslimaniofficial', 'Samy.chaffai', 'Theswaggmantv', 'Rymooshka', 'Lotfiabdelli', 'Rachabenmaouia', 'Samiramagroun', 'Amira al jaziri', 'Sami.fehri1', 'Bassoum hamraoui', 'Rymsaidi', 'Myriemboukadida', 'Saberrebai', 'Officialnajlaettounsia', 'Nordo officiel', 'Ramla dhouibi', 'A.l.a official', 'Cheb bachir1', 'Oumaymabenhafsia4', 'Bayazardi.officiel', 'Fatma.bououn', 'Raniatoumi official', 'Lety  lo', 'Maroua issa', 'Mastersina official', 'Lady.samara', 'Najlabenabdallah.official', 'Ghada.chaka', 'Lynda toumy', 'Tarek  baalouch', 'Hanen chograni', 'Emnafakher', 'Yassinebengamra', 'Onsjabeur', 'Chirinelajmi', 'Faycellahdiri', 'Mustafa hou